Install Requirements ()

In [1]:
!pip install autogen-agentchat autogen-ext[openai] -q
!pip install groq -q
!pip install arxiv -q
!pip install PyPDF2 -q

In [2]:
from autogen_agentchat.agents import AssistantAgent, UserProxyAgent
from autogen_agentchat.conditions import TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console
from autogen_ext.models.openai import OpenAIChatCompletionClient
import arxiv
from autogen_core.tools import FunctionTool
from autogen_agentchat.conditions import MaxMessageTermination

In [3]:
import os
from getpass import getpass

tokenGROQ = getpass('Enter GROQ_API_KEY here: ')
os.environ["GROQ_API_KEY"] = tokenGROQ

Enter GROQ_API_KEY here: ··········


In [ ]:
from autogen_agentchat.agents import AssistantAgent, UserProxyAgent
from autogen_agentchat.conditions import TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console
from autogen_ext.models.openai import OpenAIChatCompletionClient

# Create the agents.
custom_model_client = OpenAIChatCompletionClient(
    model="llama-3.3-70b-versatile",
    base_url="https://api.groq.com/openai/v1",
    api_key=os.environ["GROQ_API_KEY"],
    model_info={
        "vision": False,
        "function_calling": True,
        "json_output": False,
        "family": "unknown",
    },
)
assistant = AssistantAgent("assistant", model_client=custom_model_client)
user_proxy = UserProxyAgent("user_proxy", input_func=input)  # Use input() to get user input from console.

# Create the termination condition which will end the conversation when the user says "APPROVE".
termination = TextMentionTermination("DONE")

# Create the team.
team = RoundRobinGroupChat([assistant, user_proxy], termination_condition=termination)

# Run the conversation and stream to the console.
stream = team.run_stream(task="the pdf file is a little bit large, can you read it if i sent it to you completely, or you need it in small chunks?")
await Console(stream)

---------- user ----------
the pdf file is a little bit large, can you read it if i sent it to you completely, or you need it in small chunks?
---------- assistant ----------
I'm a large language model, I don't have the capability to directly receive or store files, including PDFs. I can process text-based input, but I don't have a file upload or download mechanism.

If you'd like to share the content of the PDF with me, you can try copying and pasting the text into this chat window. If the text is too long, you can break it up into smaller chunks and send them to me one at a time. I'll do my best to assist you with the content.


In [ ]:
import arxiv

# Construct the default API client.
client = arxiv.Client()

# Search for the 10 most recent articles matching the keyword "quantum."
search = arxiv.Search(
  query = "Multi-agent LLM systems",
  max_results = 1,
  sort_by = arxiv.SortCriterion.SubmittedDate
)

for result in client.results(search):
  print(f"Entry ID: {result.entry_id}")
  print(f"\nTitle: {result.title}")
  print(f"\nAuthors: {', '.join([a.name for a in result.authors])}")
  print(f"\nPublished: {result.published}")
  print(f"\nUpdated: {result.updated}")
  print(f"\nSummary: {result.summary}")
  print(f"\nComment: {result.comment}")
  print(f"\nJournal Reference: {result.journal_ref}")
  print(f"\nDOI: {result.doi}")
  print(f"\nPrimary Category: {result.primary_category}")
  print(f"\nCategories: {result.categories}")
  print(f"\nPDF URL: {result.pdf_url}\n")

In [ ]:
paper = next(arxiv.Client().results(arxiv.Search(id_list=["1605.08386v1"])))
print(paper.get_short_id())

In [ ]:
import PyPDF2
import openai

# Step 1: Extract text from the PDF
pdf_path = "/content/downloaded-paper.pdf"  # Adjust the path as needed
paper_text = ""
with open(pdf_path, "rb") as f:
    reader = PyPDF2.PdfReader(f)
    for page in reader.pages:
        page_text = page.extract_text()
        if page_text:
            paper_text += page_text + "\n"

# Step 2: Split the text into manageable chunks
def chunk_text(text, max_length=2000):
    """Splits text into chunks of approximately max_length characters."""
    return [text[i:i+max_length] for i in range(0, len(text), max_length)]


chunks = chunk_text(paper_text, max_length=2000)


In [ ]:
print(chunks[1])

**1. Research Agent (Paper Retriever)**

    Purpose: Searches for and downloads relevant academic papers from databases (e.g., Google Scholar, ArXiv, Semantic Scholar).
    Responsibilities:
    1.   Querying research databases with relevant keywords.
    2.   Downloading full-text articles or abstracts.
    3.   Storing references in a structured manner (e.g., BibTeX or JSON format).

In [75]:
def arxiv_search(query: str, max_results: int = 2) -> list:  # type: ignore[type-arg]
    """
    Search Arxiv for papers and return the results including abstracts.
    """
    import arxiv

    client = arxiv.Client()
    search = arxiv.Search(query=query, max_results=max_results, sort_by=arxiv.SortCriterion.Relevance)
    print("************ calling")
    results = []
    for paper in client.results(search):
        results.append(
            {
                "title": paper.title,
            }
        )

    return results

In [76]:

arxiv_search_tool = FunctionTool(
    arxiv_search, description="Search Arxiv for papers related to a given topic, including abstracts"
)

In [77]:
arxiv_search_agent = AssistantAgent(
    name="Arxiv_Search_Agent",
    tools=[arxiv_search_tool],
    model_client = OpenAIChatCompletionClient(
      model="llama-3.3-70b-versatile",
      base_url="https://api.groq.com/openai/v1",
      api_key=os.environ["GROQ_API_KEY"],
      model_info={
        "vision": False,
        "function_calling": True,
        "json_output": False,
        "family": "unknown",
      },
    ),
    description="An agent that can search Arxiv for papers related to a given topic, including abstracts",
    system_message="You are a helpful AI assistant. Solve tasks using your tools. Specifically, you can take into consideration the user's request and craft a search query that is most likely to return relevant academi papers.",
)

**2. Reader Agent (Document Processor)**

    Purpose: Reads, parses, and extracts key information from research papers.
    Responsibilities:
    1.   Extracting abstracts, keywords, and main sections (introduction, methodology, results, discussion).
    2.   Identifying key arguments, methodologies, and findings.
    3.   Converting scanned PDFs to machine-readable text (OCR, if needed).

In [78]:
report_agent = AssistantAgent(
    name="Report_Agent",
    model_client = OpenAIChatCompletionClient(
      model="llama-3.3-70b-versatile",
      base_url="https://api.groq.com/openai/v1",
      api_key=os.environ["GROQ_API_KEY"],
      model_info={
        "vision": False,
        "function_calling": True,
        "json_output": False,
        "family": "unknown",
      },
    ),
    description="Generate a report based on a given topic",
    system_message="You are a helpful assistant. Your task is to get the papers info mostly only titles from another agent you're gonna select only recent one and give it back to the user, user might ask questions you still need to answer their questions.",
)

**5. Human Feedback Agent (Interactive Editor)**

    Purpose: Allows human users to provide feedback and modify the generated text.
    Responsibilities:
        1.   Displaying the draft for user review.
        2.   Accepting edits, comments, and suggestions.
        3.   Incorporating changes into the final version.

In [79]:
user_proxy = UserProxyAgent("user_proxy", input_func=input)

Finally, we will create a team of agents and configure them to perform the tasks.

In [80]:
text_mention_termination = TextMentionTermination("TERMINATE")
max_messages_termination = MaxMessageTermination(max_messages=25)
termination = text_mention_termination | max_messages_termination


team = RoundRobinGroupChat(
    participants=[arxiv_search_agent, report_agent, user_proxy], termination_condition=termination
)

In [82]:
await Console(
    team.run_stream(
        task="Give me name of some research papers reagrding no code tools",
    )
)

---------- user ----------
Give me name of some research papers reagrding no code tools
************ calling---------- Arxiv_Search_Agent ----------

[FunctionCall(id='call_asaz', arguments='{"query": "no code tools research papers", "max_results": 5}', name='arxiv_search')]
---------- Arxiv_Search_Agent ----------
[FunctionExecutionResult(content='[{\'title\': "Ease on Down the Code: Complex Collaborative Qualitative Coding Simplified with \'Code Wizard\'"}, {\'title\': \'Bridging the Gap: A Survey and Classification of Research-Informed Ethical Hacking Tools\'}, {\'title\': \'A Tool for In-depth Analysis of Code Execution Reasoning of Large Language Models\'}, {\'title\': \'Deep Learning-Based Video Coding: A Review and A Case Study\'}, {\'title\': \'Code Swarm: A Code Generation Tool Based on the Automatic Derivation of Transformation Rule Set\'}]', call_id='call_asaz', is_error=False)]
---------- Arxiv_Search_Agent ----------
[{'title': "Ease on Down the Code: Complex Collaborative

TaskResult(messages=[TextMessage(source='user', models_usage=None, content='Give me name of some research papers reagrding no code tools', type='TextMessage'), ToolCallRequestEvent(source='Arxiv_Search_Agent', models_usage=RequestUsage(prompt_tokens=1689, completion_tokens=26), content=[FunctionCall(id='call_asaz', arguments='{"query": "no code tools research papers", "max_results": 5}', name='arxiv_search')], type='ToolCallRequestEvent'), ToolCallExecutionEvent(source='Arxiv_Search_Agent', models_usage=None, content=[FunctionExecutionResult(content='[{\'title\': "Ease on Down the Code: Complex Collaborative Qualitative Coding Simplified with \'Code Wizard\'"}, {\'title\': \'Bridging the Gap: A Survey and Classification of Research-Informed Ethical Hacking Tools\'}, {\'title\': \'A Tool for In-depth Analysis of Code Execution Reasoning of Large Language Models\'}, {\'title\': \'Deep Learning-Based Video Coding: A Review and A Case Study\'}, {\'title\': \'Code Swarm: A Code Generation T

In [28]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.base import Handoff
from autogen_agentchat.conditions import HandoffTermination, TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console
from autogen_ext.models.openai import OpenAIChatCompletionClient

custom_model_client = OpenAIChatCompletionClient(
    model="llama-3.3-70b-versatile",
    base_url="https://api.groq.com/openai/v1",
    api_key=os.environ["GROQ_API_KEY"],
    model_info={
        "vision": False,
        "function_calling": True,
        "json_output": False,
        "family": "unknown",
    },
)

# Create a lazy assistant agent that always hands off to the user.
lazy_agent = AssistantAgent(
    "assistant",
    model_client=custom_model_client,
    system_message="help the user with his questions",
)

# Define a termination condition that checks for handoff message targetting helper and text "TERMINATE".
# handoff_termination = HandoffTermination(target="user")
text_termination = TextMentionTermination("TERMINATE")
max_messages_termination = MaxMessageTermination(max_messages=7)
combined_termination = max_messages_termination | text_termination

# Create a single-agent team.
user_proxy = UserProxyAgent("user_proxy", input_func=input)
lazy_agent_team = RoundRobinGroupChat([lazy_agent, user_proxy], termination_condition=combined_termination)

# Run the team and stream to the console.
task = "What is the weather in New York?"
await Console(lazy_agent_team.run_stream(task=task))

---------- user ----------
What is the weather in New York?
---------- assistant ----------
I'm a large language model, I don't have have access to real-time information, so I can't provide you with the current weather in New York. However, I can suggest some ways for you to find out the current weather in New York:

1. **Check online weather websites**: You can visit websites like weather.com, accuweather.com, or wunderground.com to get the current weather conditions in New York.
2. **Use a weather app**: You can download a weather app on your smartphone, such as Dark Sky or Weather Underground, to get real-time weather updates for New York.
3. **Check social media**: You can check the official social media accounts of the National Weather Service (NWS) or the New York City government to get updates on the weather.
4. **Turn on the TV**: You can watch the local news or weather channel on TV to get the current weather conditions in New York.

If you're looking for general information a

TaskResult(messages=[TextMessage(source='user', models_usage=None, content='What is the weather in New York?', type='TextMessage'), TextMessage(source='assistant', models_usage=RequestUsage(prompt_tokens=49, completion_tokens=280), content="I'm a large language model, I don't have have access to real-time information, so I can't provide you with the current weather in New York. However, I can suggest some ways for you to find out the current weather in New York:\n\n1. **Check online weather websites**: You can visit websites like weather.com, accuweather.com, or wunderground.com to get the current weather conditions in New York.\n2. **Use a weather app**: You can download a weather app on your smartphone, such as Dark Sky or Weather Underground, to get real-time weather updates for New York.\n3. **Check social media**: You can check the official social media accounts of the National Weather Service (NWS) or the New York City government to get updates on the weather.\n4. **Turn on the T